# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [12]:
from pyspark.sql import SparkSession

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import desc,col
from pyspark.sql.functions import asc
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window

In [2]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

In [3]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [4]:
#Q1) Which page did user id "" (empty string) NOT visit?
user_log.select("page").where(user_log.userId == "").dropDuplicates().sort("page").show()

+-----+
| page|
+-----+
|About|
| Help|
| Home|
|Login|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


Unregistered users probably 

# Question 3

How many female users do we have in the data set?

In [5]:
#Q3) How many female users do we have in the data set?
user_log.filter(user_log.gender == 'F')\
        .select('userId')\
        .dropDuplicates()\
        .count()

462

# Question 4

How many songs were played from the most played artist?

In [6]:
#Q4) How many songs were played from the most played artist?
user_log.filter(user_log.page == 'NextSong').groupBy('artist').count().orderBy(desc('count')).show(n=1)

+--------+-----+
|  artist|count|
+--------+-----+
|Coldplay|   83|
+--------+-----+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [13]:
#Q5) How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

